<a href="https://colab.research.google.com/github/kumar-abhishek/handson-ml2/blob/master/ECPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Algorithm

1. Take the document, split into clauses
2. Find embeddings of the clauses
3. Feed embeddings of clauses into a Bi-LSTM layer(word-level), followed by attention layer 
4. Output of previous layer gets copied into 2 components.
5. 1 component is for emotion extraction and is a Bi-LSTM layer(clause-level)
6. 2nd compoent is for cause extraction and is a Bi-LSTM layer(clause-level)
7. Loss Lp of the whole model is the weighted sum of two components:
Lp=n*Le+(1-n)*Lc
where n is a hyper-param






In [0]:
!pip install tensorflow==2.0.0


     |████████████████████████████████| 86.3MB 36kB/s 
     |████████████████████████████████| 3.8MB 46.7MB/s 
     |████████████████████████████████| 450kB 60.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=74182586624c53fa3faea26c413a71f6afeaa737cf2e685afbe272f1a5b74460
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.2
    Uninstalling tensorflow-1.15.2:
      Successfully uninstalled tensorflow-1.15.2


In [0]:
!pip install tensorflow-probability==0.8.0

     |████████████████████████████████| 2.5MB 2.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=cc89da8567f2f21ad86614c12a82b79eee9a043044a026d8af1c50c5bfc59590
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0rc2 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: gym 0.17.1 has requirement cloudpickle<1.4.0,>=1.2.0, but you'll have cloudpickle 1.1.1 which is incompatible.
  Found existing installation: cloudpickle 1.3.0
    Uninstalling cloudpickle-1.3.0:
      Successfully uninstalled cloudpickle-1.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-probability 0.9.0
    Uninstalling tensorflow-probability-0.9.0:
      Successfully uninstalled tensorflow-probability-0.9.0


In [30]:
%tensorflow_version 2.x
%tensorflow_version


Currently selected TF version: 2.x
Available versions:
* 1.x
* 2.x


In [0]:

import unicodedata
import re
import numpy as np
import os
import io
import time
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
emotion_seeds = set(["ashamed", "delighted", "pleased", "concerned", "delight", "happy", "embarrassed", "furious", "nervous", 
                     "miffed", "angry", "mad", "anger", "excitement", "horror", "resentful", "astonished", "revulsion", 
                     "frightened", "cross", "sad", "down", "astonishment", "miserable", "worried", "sorrow", "overjoyed",
                     "dismay", "grief", "annoyance", "alarmed", "astounded", "anguish", "despair", "infuriated", 
                     "embarrassment", "peeved", "amused", "disgruntled", "indignant", "thrilled", "anxious", "excited",
                     "exasperation", "petrified", "heartbroken", "saddened", "depressed", "dismayed", "frustrated", "fedup", "livid",
                     "revulsion", "bewildered", "flabbergasted", "happier", "ecstatic", "elation", "exhilarated", "exhilaration",
                     "glee", "gleeful", "crestfallen", "sadness", "amusement", "dejected", "desolate", "despondency", "horrors",
                     "agitated", "disquiet", "horrified", "exasperated", "irked", "disgruntlement", "sickened", "revolted",
                     "devastated", "heartbreak", "inconsolable", "bewilderment", "nonplussed", "puzzlement", "disquieted",
                     "glum", "downcast", "griefstricken", "startled", "disgusted"])

In [0]:
# input is sentence, output is emotion cause pairs
# Determine clauses by splitting on punctuation.

# preprocess

# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

def remove_nonascii(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()
  return w


def preprocess_sentence(w):
  w = remove_nonascii(w)  
  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [0]:
def extract_cause(text):
  cur_cause=''
  try:
    cur_cause = re.findall('<cause>(.*?)<\\\cause>', text)[0]
    # Remove tags from line
    text=re.sub('<cause>', '', text)
    text=re.sub('<\\\cause>', '', text)
  except:
    pass
  #print('here:', text)
  return (cur_cause, text)

In [0]:
def clean_filter_clauses(all_clauses):
  cause = ''
  clauses=[]
  for clause in all_clauses:
    e_cause, e_text = extract_cause(clause)
    if e_cause!='':
      cause = remove_nonascii(e_cause)
    clauses.append(remove_nonascii(e_text))
  return cause, clauses



In [0]:

path_to_file = "data.txt"  
# 1. Remove any accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [document, emotion, cause, clauses list]
document=[]
emotion=[]
cause=[]
clause=[]
def create_dataset(path, num_examples):
  document.clear()
  emotion.clear()
  cause.clear()
  clause.clear()
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
  for i, line in enumerate(lines[:num_examples]):
    cur_emotion = re.findall('<(.*?)>', line)[0]
    # removing emotion tag in document
    text_without_emotion=line[2+len(cur_emotion):len(line)-len(cur_emotion)-3]
    #document.append(text_without_emotion)
    emotion.append(cur_emotion)

    # Determine clauses by splitting on punctuation.
    all_clauses = re.split("[.,!;:\"]+", text_without_emotion)
    filter_cause, filter_clauses = clean_filter_clauses(all_clauses)
    cause.append(filter_cause)
    clause.append([filter_clauses])
    doc = extract_cause(text_without_emotion)[1]
    # clean up document
    clean_doc = preprocess_sentence(doc)
    document.append(clean_doc)

    # clean up clauses
    # TODO
  return [document, emotion, cause, clause]

In [37]:
document, emotion, cause, clause_list = create_dataset(path_to_file, 500)
print(len(document))
for i in range(5):
  print(document[i])
  print(emotion[i])
  print(cause[i])
  print(clause_list[i])
  print('\n--------\n')

500
<start> i suppose i am happy , being so tiny it means i am able to surprise people with what is generally seen as my confident and outgoing personality . <end>
happy
being so tiny
[['i suppose i am happy', 'being so tiny', 'it means i am able to surprise people with what is generally seen as my confident and outgoing personality', '']]

--------

<start> lennox has always truly wanted to fight for the world title and was happy , because he was taking the tough route . <end>
happy
because he was taking the tough route
[['lennox has always truly wanted to fight for the world title and was happy', 'because he was taking the tough route', '']]

--------

<start> he was a professional musician now , still sensitive and happy , doing something he loved . <end>
happy
doing something he loved
[['he was a professional musician now', 'still sensitive and happy', 'doing something he loved', '']]

--------

<start> holmes is happy , because , he has the freedom of the house when we are out . <

In [49]:
X=document
#X=[tf.constant(sentence) for sentence in document]
#print(X[0], X[0].shape, (X[0].numpy()))
y=emotion
print(y)

['happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 

In [0]:
tokenizer_emotion=keras.preprocessing.text.Tokenizer(num_words=10000, oov_token="xxxxxxx")
tokenizer_emotion.fit_on_texts(emotion_seeds)


In [0]:
tokenizer=keras.preprocessing.text.Tokenizer(num_words=10000, oov_token="xxxxxxx")
tokenizer_emotion=keras.preprocessing.text.Tokenizer(num_words=10000, oov_token="xxxxxxx")
def use_tokenizer():
  tokenizer.fit_on_texts(X)
  tokenizer_emotion.fit_on_texts(emotion_seeds)
  X_dict=tokenizer.word_index

  X_seq=tokenizer.texts_to_sequences(X)
  X_padded_seq=pad_sequences(X_seq,padding='post',maxlen=40) 
  print(X_padded_seq[:3], X_padded_seq.shape, type(X_padded_seq))
  print(X_padded_seq.shape)
  
  return X_padded_seq

In [107]:
X_padded_seq = use_tokenizer()

[[  3  10 450  10  56  46  40  30 825  21 451  10  56 246   5 452 113  15
   57  28 453 308  31  43 826   8 827 828   2   0   0   0   0   0   0   0
    0   0   0   0]
 [  3 829  52 211 830 212   5 831  19   4 114 832   8   9  46 115  11   9
  309   4 833 834   2   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  3  11   9  12 454 835  73  96 836   8  46 310 837  11 311   2   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]] (500, 40) <class 'numpy.ndarray'>
(500, 40)


In [108]:
y=tokenizer_emotion.texts_to_sequences(y)
y=np.array(y)

AttributeError: ignored

In [109]:
print(y.shape)

(500, 1)


In [125]:
import tensorflow_hub as hub
from keras.models import Model, Sequential

clauses = keras.layers.Input(shape=X_padded_seq.shape[1:])
clauses_embeddings = keras.layers.Embedding(input_length=40,input_dim=10000,output_dim=50) (clauses)
output1 = keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True))(clauses_embeddings)

attention_output = keras.layers.Attention()([output1, output1, output1]) # no idea why we use output3, 3 times TODO!!!

output_emotion = keras.layers.Bidirectional(keras.layers.LSTM(64))(attention_output)
output_emotion_dense1 = keras.layers.Dense(128, activation="relu") (output_emotion)
output_emotion_dense2 = keras.layers.Dense(len(tokenizer_emotion.word_index), activation='softmax') (output_emotion_dense1)


output_cause = keras.layers.Bidirectional(keras.layers.LSTM(64))(attention_output)
output_cause_dense1 = keras.layers.Dense(128, activation="relu") (output_cause)
output_cause_dense2 = keras.layers.Dense(len(tokenizer_emotion.word_index), activation='softmax') (output_cause_dense1)

model = keras.Model(inputs=clauses, outputs=[output_emotion_dense2, output_cause_dense2])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
model.fit(X_padded_seq, y, epochs=2)

#loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)


Epoch 1/2
16/16 [==============================] - 0s 26ms/step - loss: 7.2910 - dense_54_loss: 3.5418 - dense_56_loss: 3.7492 - dense_54_accuracy: 0.3760 - dense_56_accuracy: 0.3980
Epoch 2/2
16/16 [==============================] - 0s 24ms/step - loss: 2.8390 - dense_54_loss: 1.3489 - dense_56_loss: 1.4900 - dense_54_accuracy: 0.4240 - dense_56_accuracy: 0.3520


In [0]:
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)


In [103]:
# this piece of code just takes document as input and predicts emotion as the output of the document
text_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_length=40,input_dim=10000,output_dim=50),
    #tf.keras.layers.Flatten(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    #tf.keras.layers.Attention(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(len(tokenizer_emotion.word_index), activation='softmax')
])

text_model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

text_model.summary()

print(X_padded_seq.shape, y.shape)
text_model.fit(X_padded_seq, y, epochs=5)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 40, 50)            500000    
_________________________________________________________________
bidirectional_83 (Bidirectio (None, 128)               58880     
_________________________________________________________________
dense_33 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_34 (Dense)             (None, 89)                11481     
Total params: 586,873
Trainable params: 586,873
Non-trainable params: 0
_________________________________________________________________
(500, 40) (500, 1)
Epoch 1/5
16/16 [==============================] - 0s 10ms/step - loss: 4.2503 - accuracy: 0.3500
Epoch 2/5
16/16 [==============================] - 0s 10ms/step - loss: 1.6925 - accuracy: 0.4220
Epoch 3/5
16/16 [==============

In [87]:
# test code to test the model prediction
test_input=['he was mad']
test_X_seq=tokenizer.texts_to_sequences(test_input)
print(test_X_seq)

test_X_padded_seq=pad_sequences(test_X_seq,padding='post',maxlen=40)
print(test_X_padded_seq)

output = text_model.predict_classes(test_X_padded_seq)
print(output)
for word, index in tokenizer_emotion.word_index.items():
  if index==output:
    print(word, index, output)
    break

[[11, 9, 85]]
[[11  9 85  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
[68]
happy 68 [68]


Questions:
1. Do we need seeding the model?
2. HW:
 a. Use pretrained embeddings
 **b. use functional apis**
 c. Limit output to the emotions set(say 10)
 d. Look at other sources of data(emotion-cause or emotion-entailments)


